# Investigating Ray Tune hyperparameter tuning runs

This notebook uses Ray Tune's built-in search visualisation tools to show you how well tuning is doing, which hyperparameters are important/unimportant, etc. I suggest pointing it to your running Ray Tune search & regularly running it to make sure that the search is making progress. You may need to periodically interrupt your search & restart with tighter hyperparameter search ranges if you find that some hyperparameters are consistently terrible.

In [ ]:
%matplotlib inline

import glob
import html
import io
import numbers
import os

import cloudpickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ray
import seaborn as sns
import skopt

from IPython.display import display, Image, HTML
from ray import tune
from skopt.plots import plot_evaluations, plot_objective

from il_representations.utils import load_sacred_pickle

sns.set(context='notebook', style='darkgrid')

# !pip uninstall -y scipy sckikit-learn
# !pip install scikit-learn==0.24.1 'scipy>=1.6'

## Variables that you can configure

In [ ]:
# Directory used by the running Ray Tune instance. Should contain a file named
# experiment_state-<date>.json.
# *THIS SHOULD BE THE ONLY THING YOU NEED TO CHANGE*
# Example: RUNNING_RAY_TUNE_DIR = '../runs/chain_runs/53/grid_search/'
RUNNING_RAY_TUNE_DIR = '/scratch/sam/il-representations-gcp-volume/cluster-data/cluster-2021-01-25-tuning-try1/chain_runs/2/grid_search/'

## Loading internal scikit-optimise experiment state

In [ ]:
class WrappedConfig:
    """Dumb wrapper class used in pretrain_n_adapt to hide things from skopt.
    It's in a separate module so that we can pickle it when pretrain_n_adapt is
    __main__."""
    def __init__(self, config_dict):
        self.config_dict = config_dict


search_alg_pattern = os.path.join(RUNNING_RAY_TUNE_DIR, 'search-alg-*.pkl')
pickle_paths = glob.glob(search_alg_pattern)
if not pickle_paths:
    raise IOError(
        "Could not find any matches for skopt state pattern, "
        f"{search_alg_pattern!r}. Check whether skopt's .pkl file actually "
        f"exists in RUNNING_RAY_TUNE_DIR={RUNNING_RAY_TUNE_DIR!r}.")
pickle_path, = pickle_paths
with open(pickle_path, 'rb') as fp:
    _, skopt_alg = load_sacred_pickle(fp)
    
skopt_res = skopt_alg.get_result()
    
# If variable names have not been saved and you have to add them back in, you can do something like this:
# variable_names = ['foo', 'bar', 'baz', 'spam', 'ham', 'asdf']
# for var_name, var in zip(variable_names, skopt_alg.space.dimensions):
#     var.name = var_name

## Inferring the task name

We keep the "base config"—which is shared between all tuning runs—in a special skopt categorical variable in order to get around a Ray Tune bug. Helpfully, this lets us infer the benchmark and task name (assuming that those variables aren't also being optimised over by skopt).

In [ ]:
dims_by_name = {d.name: d for d in skopt_alg.space.dimensions}
base_cfg_outer, = dims_by_name['+base_config'].categories
base_cfg_inner = base_cfg_outer.config_dict
env_cfg = base_cfg_inner['env_cfg']
bench_name = env_cfg['benchmark_name']
task_name = env_cfg['task_name']
exp_ident = base_cfg_inner['il_test']['exp_ident']
# magic spans are so that Google Docs doesn't mangle the bold :(
display(HTML(f"""<p>
<br/>
<hr>
<span style='font-weight: normal;'>Basic</span> config task/environment is <strong>{html.escape(bench_name)}</strong>/<strong>{html.escape(task_name)}</strong>.<br/>
There are <strong>{len(skopt_alg.Xi)}</strong> completed runs in this hyperparameter tuning file.<br/>
<code>exp_ident = <strong>{exp_ident}</strong></code><span style='font-weight: auto;'>.</span><br/><br/>
</p>"""))

## Generating hyperparameter sensitivity plots

In [ ]:
def force_display_figure(figure=None):
    """Sometimes Jupyter refuses to show the most recently plotted figure.
    This helper can _force_ it to display the figure.
    
    (unfortunately the Jupyter failure is sporadic, so we can't _always_
    call this function or else we end up with double plots)"""
    if figure is None:
        figure = plt.gcf()
    with io.BytesIO() as fp:
        figure.savefig(fp)
        fig_data = fp.getvalue()
    display(Image(fig_data))

# If we leave plot_dims out, then skopt tries to infer plot_dims itself.
# Unfortunately skopt 0.8.1 has broken code for skipping over constant dimensions
# (see the "if space.dimensions[row].is_constant:" branch in plot_evaluations---it
# fails to update other arrays when it omits the constant dimension from plot_dims).
# Thus we manually provide all plot_dims ourselves.
_ = plot_evaluations(skopt_res, plot_dims=[d.name for d in skopt_res.space.dimensions])
# force_display_figure()

_ = plot_objective(skopt_res, n_samples=40, minimum='expected_minimum_random', n_minimum_search=1000)

n_results = len(skopt_res.func_vals)
fig = plt.figure()
sns.distplot(skopt_res.func_vals, rug=True, norm_hist=False, kde=False, bins=10 if n_results >= 20 else None)
plt.title(f"Final loss distribution from {n_results} runs (lower = better)")
plt.xlabel("Final loss")
plt.ylabel("Frequency")
# force_display_figure(fig)

## Listing the best encountered hyperparameter settings, ordered by loss

In [ ]:
# we plot any config that has loss below 'thresh'
# (by default, I've made it show the top 10 best configs;
# you can change 'thresh' to anything you want)
thresh = max(sorted(skopt_res.func_vals)[:20])
good_inds, = np.nonzero(skopt_res.func_vals <= thresh)
# for conf_num, good_ind in enumerate(good_inds, start=1):
#     print(
#         f"Good config at index {good_ind} ({conf_num}/"
#         f"{len(good_inds)}), thresh {thresh}:")
#     # TODO: print function value here too
#     all_dims = skopt_res.space.dimensions
#     for dim, value in zip(all_dims, skopt_res.x_iters[good_ind]):
#         if dim.name == '+base_config':
#             continue
#         print(f'    {dim.name} = {value}')
        
print(f'Amalgamated "good" configs at thresh {thresh}:')
for dim_idx, dimension in enumerate(skopt_res.space.dimensions):
    if dimension.name == '+base_config':
        continue
    values = [skopt_res.x_iters[i][dim_idx] for i in good_inds]
    if isinstance(values[0], float):
        values_str = f"[{', '.join('%.3g' % v for v in values)}]"
    else:
        values_str = str(values)
    if isinstance(values[0], (numbers.Number, bool)):
        values_str += f' (mean: {np.mean(values)})'
    print(f'    {dimension.name} = {values_str}')

## Getting skopt to guess which configurations are going to perform best

In [ ]:
skopt_minima = []
for i in range(10):
    skopt_min = expected_minimum_random_sampling(
        skopt_res, n_random_starts=1000000)
    skopt_minima.append(skopt_min[0])
print("skopt's guess at best configs (randomly sampled proposals):")
for idx, dim in enumerate(skopt_res.space.dimensions):
    name = dim.name
    values = [m[idx] for m in skopt_minima]
    if isinstance(values[0], float):
        stringified = [f'{v:.3g}' for v in values]
    else:
        stringified = map(str, values)
    min_str = f'  {name} = [{", ".join(stringified)}]'
    print(min_str)